In [128]:
# Google drive file ID for spreadsheet
GDID = '1k7uoC0_GWPJ2x-mMPYTEyaBXUvmaegJ_'

In [129]:
import requests
url = 'https://drive.google.com/uc?export=download&id='+GDID
myfile = requests.get(url)
open('setExcel.xlsm', 'wb').write(myfile.content);

In [130]:
# Copy Excel Sheet over to sheet that contains the macros, and run the macros
# This will create two .xml files. One in the main directory called setEons.xml, which contains the card data needed by Strange Eons.
# The second will be placed in the setOCTGN folder, which will be copied over to your OCTGN folder later in this workbook.

import xlwings as xw
app = xw.App(visible=False) # IF YOU WANT EXCEL TO RUN IN BACKGROUND
xlws = {}

xlwb1 = xw.Book('setExcel.xlsm')
xlws['ws1a'] = xlwb1.sheets['Sets']
xlws['ws1b'] = xlwb1.sheets['Card Data']

xlwb2 = xw.Book('macros.xlsm')
xlws['ws2a'] = xlwb2.sheets['Sets']
xlws['ws2b'] = xlwb2.sheets['Card Data']

newVal = xlws['ws1a'].range('A3:B3').value   # get set id/name
xlws['ws2a'].range('A3:B3').value = newVal   # update set id/name

newVal = xlws['ws1b'].range('A2:AT1000').value   # get card data
xlws['ws2b'].range('A2:AT1000').value = newVal   # update card data

# This macro will generate a folder under setOCTGN, which which will be copied over to OCTGN\GameDatabase\a21af4e8-be4b-4cda-a6b6-534f9717391f\Sets 
macro1 = xlwb2.macro('SaveOCTGN')
macro1()

# This macro generates a file called setEons.xml, which is used by the Strange Eons macros.
macro2 = xlwb2.macro('SaveXML')
macro2()

xlwb2.save()
xlwb1.close()
xlwb2.close()

### Strange Eons instructions:

Start by opening up the `setGenerator.seproject` file in the main directory.

### Now, if it's your first time opening Strange Eons for this project:

1. Open each of the files in "template" and edit the "Collection" tab for each one as needed for the project.
2. Run the makeCards script by double clicking it. This will take the templates, fill them with the text from the setEons.xml generated above, and save them under the "new" folder in Strange Eons.
3. Move all the files in "new" to "current".

### To add images:
1. Open up a file in "current"
2. Click on the Portrait tab
3. From your computer, navigate to the imagesRaw folder and drag an image from there into Strange Eons to add it to the card.
4. Within Strange Eons, move and scale the image as necessary.
5. If it's an encounter card, do the same thing with the encounter set icon with the Encounter Set tab and the imagesIcons folder.
6. Save the file, close it, and move on to the next.
7. Note: If you do any modifications to the text, they will not be preserved by the next macro. Modifications to the text should be done only in the original Google sheet.

### To update text without messing up the images:
1. Modify the Google sheet text.
2. Run the above python cells to update the setEons.xml file. 
3. Run the updateCards script in Stange Eons by double clicking it. This will take the cards in "current", which should have images now, put in the updated text from the setEons.xml generated above, and save them under the "updated" folder in Strange Eons.

### To export the cards
1. Make sure the "Bulk Export" Plug-in is installed.
2. Right-click the "updated" folder, click on "Bulk Export, choose the settings you want, and hit "Export"
3. Once complete, copy all the files in "updated" to your imagesEons folder (the .eon file too if you want).

### Important: Dealing with name, id, etc. changes
Since the Card Number, Quantity, Name, and OCTGN ID all appear in the file name, any changes to these values will result in a desyncronization when trying to run updateCards. Here are the steps to follow in this case:
1. In the "current" Strange Eons folder, manually fix the relevant file name.
2. If your modification resulted in a longer filename, remove some "-" characters so that the total number of characters in the file name stays constant.
3. Alternatively, if your modification resulted in a shorter filename, add extra "-" characters so that the total number of characters in the file name stays constant.
4. Then, run the updateCards script.

## Update OCTGN

In [102]:
import os
# Directory of OCTGN installation
dirOCTGN = os.path.expanduser('~')+'\\Documents\\OCTGN\\'
if not os.path.isdir(dirOCTGN): print('Invalid path!')

In [113]:
#Make Image Pack
import os.path
import zipfile
import shutil

# Move files from startpath to newpath, then scan through files, processes them, and add them to a zip file to create an image pack
def zipdir(startpath, newpath, ziph):
    # Move files in current directory to new directory
    for root, dirs, files in os.walk(startpath):
        for file in files:
            oldfile = startpath+'/'+file
            if (file.endswith('.png') and os.path.isfile(oldfile)):
                newfile=newpath+file
                shutil.copyfile(oldfile, newfile)
                
    # Zip files in new directory
    for root, dirs, files in os.walk(newpath):
        for file in files:
            if file.endswith('-1.png'):
                shutil.move(newpath+file, newpath+file.replace('-1.png', '.png'))
                file = file.replace('-1.png', '.png')
            if file.endswith('-2.png'):
                shutil.move(newpath+file, newpath+file.replace('-2.png', '.B.png'))
                file = file.replace('-2.png', '.B.png')
            if (file.endswith('.png')):
                if len(file)>50: 
                        shutil.move(newpath+file, newpath+file[50:])
                        file = file[50:]
                ziph.write(os.path.join(root, file))


# Current directory
cwd = os.getcwd()

# OCTGN Set ID obtained by looking at the folders generated by the excel SaveOCTGN script
octgnids = os.listdir(cwd+'/setOCTGN')
if len(octgnids) == 0:
    print("No set folder found in ./setOCTGN. Did you run the excel script?")
else:
    for octgnid in octgnids:
        # Make directory tree to store images that will be converted to an image pack
        newpath = ".\\imagesOCTGN\\a21af4e8-be4b-4cda-a6b6-534f9717391f\\Sets\\"+octgnid+"\\Cards\\"
        if not os.path.exists(newpath):
            os.makedirs(newpath)

        # Remove any existing image pack
        fname = 'imagePack_'+octgnid+'.o8c'
        if os.path.isfile(fname): os.remove(fname)

        # Make a zip file
        zipf = zipfile.ZipFile(fname, 'w', zipfile.ZIP_DEFLATED)

        # Add images to the zip file
        print('Working...')
        zipdir('./imagesEons',newpath, zipf)
        print('Finished.')

        # Close the zip file
        zipf.close()
        
        # Now there should be an imagePack_<octgnid>.o8c file in the main directory. 
        # Open that using the "Add Image Packs" button from within OCTGN to add it.


Working...
Finished.


In [109]:
# Copy set folder to OCTGN directory
from distutils.dir_util import copy_tree
setspath = dirOCTGN+'\\GameDatabase\\a21af4e8-be4b-4cda-a6b6-534f9717391f\\Sets\\'
if os.path.isdir(dirOCTGN):
    if os.path.isdir('.\\setOCTGN\\'+octgnid):
        copy_tree('.\\setOCTGN\\'+octgnid, setspath+octgnid)
        print('Copied successfully.')
    else:
        print('No folder found to move.')
else:
    print('OCTGN Sets folder not found.')

Copied successfully.
